In [54]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

caminho_driver_64 = r"C:\Users\berna\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe"
options = webdriver.ChromeOptions()
options.binary_location = r"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
options.add_experimental_option("prefs", {
  "download.default_directory": r"C:\Users\berna\Downloads",
  "download.prompt_for_download": False,
  "download.directory_upgrade": True,
  "safebrowsing.enabled": True
})

#servico = Service(caminho_driver_64)
#navegador = webdriver.Chrome(service=servico, options=options)

In [1]:
import re

def limpar_preco(preco_str):
    # Encontra o padrão de número no formato brasileiro (ex: "4.679,10" ou "2.399,00")
    numeros = re.findall(r"\d+\.\d+,\d+|\d+,\d+", preco_str)
    
    if numeros:
        preco_limpo = numeros[0].replace(".", "").replace(",", ".")  # Remove o ponto e troca a vírgula
        return float(preco_limpo)  # Converte para número inteiro
    
    return None  # Retorna None se não encontrar um número válido

def formatar_moeda(valor):
    return f'R$ {valor:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.')


In [56]:
import pandas as pd

tabela_produtos = pd.read_excel('buscas.xlsx')
display(tabela_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 15 128gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [57]:
def buscar_google_shopping(produto):
    servico = Service(caminho_driver_64)
    navegador = webdriver.Chrome(service=servico, options=options)
    nome = f'"{produto["Nome"]}"'
    termos_banidos = produto["Termos banidos"]
    navegador.get("https://shopping.google.com/?nord=1&pli=1")
    navegador.find_element(By.XPATH, r'//*[@id="REsRA"]').send_keys(nome + " - " + termos_banidos)
    navegador.find_element(By.XPATH, r'//*[@id="REsRA"]').send_keys(Keys.ENTER)
    lista = navegador.find_elements(By.CLASS_NAME, "i0X6df")
    dict_produtos = {}
    for produto in lista:
        preco_str = produto.find_element(By.CLASS_NAME, "kHxwFf").text
        link = produto.find_element(By.TAG_NAME, "a").get_attribute('href')
        preco = limpar_preco(preco_str)
        dict_produtos[preco] = link
        
    navegador.quit()
    return dict_produtos



In [58]:
def buscar_buscape(produto):
    servico = Service(caminho_driver_64)
    navegador = webdriver.Chrome(service=servico, options=options)
    nome = produto["Nome"]
    termos_banidos = produto["Termos banidos"]
    navegador.get("https://www.buscape.com.br/")
    navegador.find_element(By.XPATH, r'/html/body/div[1]/main/header/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(nome)
    navegador.find_element(By.XPATH, r'/html/body/div[1]/main/header/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(Keys.ENTER)
    lista = navegador.find_elements(By.CLASS_NAME, "Hits_ProductCard__Bonl_")
    dict_produtos = {}
    for produto in lista:
        nome_site = produto.find_element(By.TAG_NAME, "h2").text
        if termos_banidos not in nome_site:
            preco_str = produto.find_element(By.XPATH, './/p[@data-testid="product-card::price"]').text
            preco = preco = limpar_preco(preco_str)
            link_elemento = produto.find_element(By.XPATH, './/a')  # Busca especificamente um <a>
            link = link_elemento.get_attribute("href")  # Pega o href do <a>
            dict_produtos[preco] = link
        
    navegador.quit()
    return dict_produtos


In [70]:
def buscar_produtos(tabela):
    tabela_resposta = pd.DataFrame(columns=["Nome", "Preço", "Link"])
    for indice in tabela.index:
        produto = tabela.loc[indice]
        dict_produto = {}
        dict_google = buscar_google_shopping(produto)
        dict_buscape = buscar_buscape(produto)
        dict_produto.update(dict_google)
        dict_produto.update(dict_buscape)
        nome = produto["Nome"]

        for preco in dict_produto.keys():
            if preco > produto["Preço mínimo"] and preco < produto["Preço máximo"]:
                nova_linha = {
                    "Nome": nome,
                    "Preço": formatar_moeda(preco),
                    "Link": dict_produto[preco],
                }
                
                tabela_resposta.loc[len(tabela_resposta)] = nova_linha
        
    return tabela_resposta

buscar_produtos(tabela_produtos)

,Nome,Preço,Link
0,iphone 15 128gb,"R$ 3.499,00",https://www.google.com/url?url=https://www.kab...
1,iphone 15 128gb,"R$ 3.176,10",https://www.google.com/url?url=https://www.ame...
2,iphone 15 128gb,"R$ 3.239,86",https://www.buscape.com.br/celular/smartphone-...
3,iphone 15 128gb,"R$ 3.179,00",https://www.buscape.com.br/celular/smartphone-...
4,iphone 15 128gb,"R$ 3.455,36",https://www.buscape.com.br/celular/smartphone-...
5,rtx 3060,"R$ 4.199,99",https://www.google.com/shopping/product/114018...


In [74]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import sys
from email.mime.application import MIMEApplication
from datetime import date
sys.path.append('D:\Codigos\senhas')

from senha_gmail import senha_gmail

def enviar_email_gerente():    

    hoje = date.today().strftime("%d/%m/%Y")
    msg = MIMEMultipart()
    msg["Subject"] = f"Produtos dentro da faixa de preço desejada no dia {hoje}"
    msg["From"] = "bernardobernstorff@gmail.com"
    msg["To"] = "bernardo.torff@gmail.com"
    
    tabela_produtos_html = buscar_produtos(tabela_produtos).to_html(escape=False, index=False)

    corpo_email = f"""<p>Boa dia, Bernardo</p>
    <p>O resultado da busca de produtos dentro da faixa de preço do dia {hoje} foi</p>
    <p>{tabela_produtos_html}</p>
    <p>Att., Bernardo</p>"""

    msg.attach(MIMEText(corpo_email, "html"))

    servidor = smtplib.SMTP("smtp.gmail.com", 587)
    servidor.starttls()
    servidor.login(msg["From"], senha_gmail)
    servidor.send_message(msg)
    servidor.quit()
    print("Email enviado")

    

enviar_email_gerente()

<>:8: SyntaxWarning: invalid escape sequence '\C'
<>:8: SyntaxWarning: invalid escape sequence '\C'
C:\Users\berna\AppData\Local\Temp\ipykernel_12496\3209468278.py:8: SyntaxWarning: invalid escape sequence '\C'
  sys.path.append('D:\Codigos\senhas')


Email enviado
